In [1]:
from modeling.compute import Var
from foundations.functionals import Functional
from foundations.functionals import Projectable
from foundations.functionals import Problem
from foundations.functionals import residual_solver
from foundations.functionals import generate_opt_objects
import numpy as np
from sympy import exp

In [2]:
z1,z2,x,y2 = Var('z_1'), Var('z_2'), Var('x'), Var('y_2')

In [3]:
F1 = Functional()
a = F1.Var('a', z1+z2-0.2*y2) # intermediate variable for demonstration purposes
y1 = F1.Var('y_1', x**2+a)
F2 = Functional(y2, abs(y1)**.5+z1+z2)

In [4]:
F = Functional()
F.add_encoded_functions(F1, F2)
F.projectable = Projectable(
    F1.projectable.encoded_functions
    +F2.projectable.encoded_functions)
F.f = residual_solver(F)

In [5]:
random_generator = np.random.default_rng(seed=41).random

In [6]:
xin = {x:0, z1:0, z2:0}
y = F.dict_in_dict_out(xin, 
               random_generator=random_generator)

In [7]:
y

{a: -0.04000000000000001, y_1: -0.04000000000000001, y_2: 0.2}

In [8]:
F1.dict_in_dict_out({**xin, **y})

{a: -0.04000000000000001, y_1: -0.04000000000000001}

In [9]:
F2.dict_in_dict_out({**xin, **y})

{y_2: DeviceArray(0.2, dtype=float64)}

In [10]:
obj = x**2+z2+y1+exp(-y2)
ineq = [3.16-y1, y2-24] # technically represent residual sets
# eq = [F1.set, F2.set] # using residuals for F1 and F2
problem = Problem(obj, ineq, eliminate=F, 
                bounds = {x:[0,10], z1:[0,10], z2:[0,10]}) 

In [11]:
opt_func = generate_opt_objects(problem, problem.obj, 
                     problem.ineqs, problem.eqs, problem.eliminate)

In [12]:
xin = {x:1, z1:1, z2:1}
constant_generator = lambda : 1
y = F.dict_in_dict_out(xin, 
               random_generator=constant_generator)

In [13]:
constant_generator = lambda : .2
eval_obj_constraints, x0 = opt_func(
    x_initial=xin, 
    random_generator=random_generator)

In [14]:
from scipy.optimize import minimize, NonlinearConstraint

In [15]:
obj_function = lambda x: eval_obj_constraints(x)[0]
ineq_function = lambda x: eval_obj_constraints(x)[1][:]
constraints = (NonlinearConstraint(ineq_function, 
                                   -np.inf,0),)

In [16]:
obj_function(np.array([0,0,0]))

0.7787307530779818

In [17]:
eval_obj_constraints(np.array([0,0,0]))

(0.7787307530779818, array([  3.2, -23.8]), array([], dtype=float64))

In [20]:
minimize(obj_function, x0, constraints=constraints, bounds=(
    (0,10),(0,10),(0,10)))

     fun: 3.1620869562729936
     jac: array([1.75486168e+00, 5.96046448e-08, 7.54861712e-01])
 message: 'Optimization terminated successfully'
    nfev: 32
     nit: 8
    njev: 8
  status: 0
 success: True
       x: array([0.00000000e+00, 2.25490970e-16, 4.39440972e+00])

In [21]:
eval_obj_constraints(x0)

(DeviceArray([3.16630697], dtype=float64),
 array([-4.23411544e-03, -1.78211706e+01]),
 array([], dtype=float64))

In [50]:
problem.obj.dict_in_only({**xin,**y})

DeviceArray(2.86504457, dtype=float64)

In [48]:
F1.dict_in_only({x:0., z1:4.39, z2:0, 
                          y1:3.16, y2:6.17})

(3.1559999999999997, 3.1559999999999997)

In [28]:
F2.dict_in_only({x:0., z1:4.39, z2:0, 
                          y1:3.16, y2:6.17})

(DeviceArray(6.16763888, dtype=float64, weak_type=True),)

In [51]:
problem.dict_in_dict_out({}, 
                         x_initial={x: 1, z1: 1, z2: 1},
                         random_generator=random_generator)

{x: 1.0, z_2: 1.0, z_1: 1.0}

In [11]:
problem.dict_in_dict_out({}, 
                         x_initial={x: 1, z1: 1, z2: 1},
                         random_generator=random_generator)

{z_2: 1.0, x: 1.0, z_1: 1.0}

In [15]:
F.dict_in_only({x:1, z1:2, z2:3, y1:1, y2:0})

(DeviceArray(6., dtype=float64, weak_type=True),)

In [6]:
F.projectable

() <- (a, y_2, x, z_2)
() <- (y_1, a, z_1)
() <- (y_1, y_2, z_1, z_2)

In [8]:
random_generator = np.random.default_rng(seed=42).random
outdict = F.dict_in_dict_out({x:1, z1:1, z2:1}, 
                   x_initial={y1:1, y2:1},
                   random_generator=random_generator)

In [13]:
dict(map(lambda x: (x[0],round(x[1],2)), outdict.items()))

{a: 1.3, y_1: 2.3, y_2: 3.52}

In [4]:
F1.projectable.dict_in_flat_out({z1:1, z2:2, x:3, y2:4, y1:5, a:6})

array([ 1.8, -2. ])

In [6]:
F1.dict_in_dict_out({z1:1, z2:2, y1:3, y2:4, x:5})

{a: 6.2, y_1: 7.2}

In [ ]:
F1 = Functional()
a = F1.Var('a', z2+x-0.2*y2) # intermediate variable for demonstration purposes
y1 = F1.Var('y_1', z1**2+a)
F2 = Functional(y2, y1**.5+z1+z2)

#F = F2.eliminate(F1) # eliminates 'a' and 'y_1'
F = intersection(F1, F2) # eliminates 'y_1' and 'y_2'

obj = x**2+z2+y1+exp(-y2)
ineq = [3.16-y1, y2-24] # technically represent residual sets
# eq = [F1.set, F2.set] # using residuals for F1 and F2
problem = Problem(obj, ineq, eliminate=F, 
                {'x':[0,10], 'z_1':[0,10], 'z_2':[0,10]}) # bounds

problem.solve(x0={x:1, z1:5, z2:2}) # or seed=10 for random guess

In [3]:
F1 = FunctionalComp(Component.fromsympy(
    z2+x-0.2*y2, a, arg_mapping=True))
F2 = FunctionalComp(Component.fromsympy(
    z1**2+a, y1, arg_mapping=True))
F3 = FunctionalComp(Component.fromsympy(
    y1**0.5+z1+z2, y2, arg_mapping=True))
obj = Component.fromsympy(
    x**2+z2+y1+exp(-y2), arg_mapping=True)
ineq1 = Component.fromsympy(
    3.16-y1, arg_mapping=True)
ineq2 = Component.fromsympy(
    y2-24, arg_mapping=True)

In [4]:
functionals = [F1, F2, F3]
S = ResidualSet(sum([functional.projectable.components 
                     for functional in functionals], []))
independents = sum([functional.independent for functional in functionals],())
projectto = tuple(set(elt for functional in functionals 
               for elt in functional.projected if elt not in independents))

In [5]:
y = {x:1, z2:1, z1:1}

In [6]:
F = S.project(projectto)
sol = F.solve(y)

In [7]:
J = lambda y: obj.evaldict({**y, **F.solve(y)})

In [8]:
J({x:4, z2:2, z1:-1})

DeviceArray(24.32798941, dtype=float64)

In [9]:
z = {**y, **sol}

In [10]:
F1.solve(z), F2.solve(z), F3.solve(z)

({a: DeviceArray(1.29689011, dtype=float64)},
 {y_1: DeviceArray(2.29689011, dtype=float64)},
 {y_2: DeviceArray(3.51554944, dtype=float64)})

In [11]:
obj.evaldict(z), ineq1.evaldict(z), ineq2.evaldict(z)

(DeviceArray(4.32662157, dtype=float64),
 0.8631098884280703,
 -20.48445055785965)